In [2]:
# Imports
import pandas as pd
import numpy as np
from data import Database
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import OrdinalEncoder
from torch.utils.data import random_split, TensorDataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, ReLU, Dropout
from tensorflow.keras.activations import sigmoid, softmax
from sklearn.preprocessing import StandardScaler

2023-04-27 20:38:32.072766: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
db = Database()

In [35]:
db.reset()
db.seed(300)

In [4]:
df = db.dataframe()
enc = OrdinalEncoder()
df["Rarity"] = enc.fit_transform(df[["Rarity"]])

# Split Data

In [10]:
X = df[["Level", "Health", "Energy", "Sanity"]]
y = df["Rarity"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

# Establish Baseline
baseline score = 0.34

In [38]:
y_train.value_counts(normalize=True)[0]

0.26666666666666666

# Random Forest
best score = 0.88

In [11]:
# instantiate class
rf_pipe = Pipeline([
    ("scale",StandardScaler()),
    ("clf", RandomForestClassifier())
])

# param dict for grid search
rf_param_dict = {
    "clf__n_estimators" : [80, 100, 120],
    "clf__min_samples_split" : [2, 4]
}

# instantiate and fit GridSearch
rf_grid = GridSearchCV(rf_pipe, rf_param_dict)
rf_grid.fit(X_train, y_train)
rf_grid.best_estimator_.score(X_test, y_test)

0.8833333333333333

In [16]:
rf_grid.predict_proba(X_test)[0].max()

0.79

In [35]:
rf_grid.best_params_

{'clf__min_samples_split': 2, 'clf__n_estimators': 120}

In [1]:
rf_grid.best_estimator_.predict_proba(X_test)

NameError: name 'rf_grid' is not defined

# Multinomial Logistic Regression
best score = .56

In [22]:
lr_model = LogisticRegressionCV(multi_class="ovr", cv=5, random_state=42)

In [23]:
grid_params = {
    "Cs" : [8, 10, 12],
    "cv" : [3, 5, 7]
}

lr_grid = GridSearchCV(lr_model, param_grid=grid_params)
lr_grid.fit(X_train, y_train)

GridSearchCV(estimator=LogisticRegressionCV(cv=5, multi_class='ovr',
                                            random_state=42),
             param_grid={'Cs': [8, 10, 12], 'cv': [3, 5, 7]})

In [24]:
lr_grid.best_score_

0.6125

In [25]:
lr_grid.score(X_test, y_test)

0.5666666666666667

# Tensorflow
best score = .88

In [16]:
tf = Sequential()
tf.add(Dense(10, input_dim=4, activation="relu"))
tf.add(Dense(20, activation="relu")),
tf.add(Dense(10, activation="relu")),
tf.add(Dense(6, activation="softmax"))

stop = EarlyStopping(monitor="loss", patience=3)

# compile model
tf.compile(loss="sparse_categorical_crossentropy",
           optimizer="nadam",
           metrics=["accuracy"])

In [17]:
tf.fit(X_train, y_train, batch_size=20, epochs=200, callbacks=[stop])

Epoch 1/200
12/12 [==============================] - 1s 2ms/step - loss: 2.5267 - accuracy: 0.2542
Epoch 2/200
12/12 [==============================] - 0s 2ms/step - loss: 2.1100 - accuracy: 0.2458
Epoch 3/200
12/12 [==============================] - 0s 2ms/step - loss: 1.7744 - accuracy: 0.2500
Epoch 4/200
12/12 [==============================] - 0s 2ms/step - loss: 1.5518 - accuracy: 0.4250
Epoch 5/200
12/12 [==============================] - 0s 2ms/step - loss: 1.4261 - accuracy: 0.5958
Epoch 6/200
12/12 [==============================] - 0s 2ms/step - loss: 1.3367 - accuracy: 0.5375
Epoch 7/200
12/12 [==============================] - 0s 2ms/step - loss: 1.2799 - accuracy: 0.5458
Epoch 8/200
12/12 [==============================] - 0s 2ms/step - loss: 1.2200 - accuracy: 0.5875
Epoch 9/200
12/12 [==============================] - 0s 2ms/step - loss: 1.1486 - accuracy: 0.6250
Epoch 10/200
12/12 [==============================] - 0s 2ms/step - loss: 1.0733 - accuracy: 0.6333
Epoch 11/

In [18]:
tf.evaluate(X_test, y_test)

2/2 [==============================] - 0s 5ms/step - loss: 0.3481 - accuracy: 0.8833


[0.34806981682777405, 0.8833333253860474]

# Conclusion

My TensorFlow neural network and random forest classifier tied for best model. I will use the random forest model for the app since it is simpler and I am unsure of how to get a confidence score for a neural network. The random forest was ok to begin with but improved from increasing n_estimators.